# PyWheat

Python library for simulation of wheat phenological development, crop growth and yield at large scales.

Author:
* **Ernesto Giron Echeverry** (Independent Researcher, e.giron.e@gmail.com)

Last updated: October 16, 2023


<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="https://colab.research.google.com/drive/1jgxcfLBfNNKsijy0EPRO6qXSR3wZKobE?usp=sharing">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href=
   "https://github.com/egiron/pywheat/blob/main/notebooks/Using_data_from_ERA5_and_GEE.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

In [1]:
# Install library
#!pip install pywheat

## Load Libraries

In [ ]:
import os, sys, gc
import pandas as pd
pd.set_option('display.max_columns', None)

# Load PyWheat library
import pywheat
from pywheat.data import satellite
from pywheat.pheno import determine_phenology_stage

print(pywheat.__version__)


## Load daily weather data from ERA5 using GEE

Using GEE allows you to extract weather data in an easy way from any point of the world.

In [ ]:
# Load GEE libraries and authenticate with your credentials
# Further info at https://developers.google.com/earth-engine/guides/access
# https://developers.google.com/earth-engine/guides/python_install
import ee
ee.Authenticate()
#ee.Initialize()
ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

In [ ]:
# Using data from ERA5 and GEE
# Define the initial parameters for the site
sowing_date = '2019-01-28'
lng, lat = 83.45, 27.5

# Get daily weather data using PyWheat API
weather_data = satellite.getERA5(init_dates=[sowing_date], coords=[[lng,lat]],
                       agg='DAILY_AGGR', buffer=1, dispFig=True)

# re-format or rename columns to match with PyWheat structure
weather_data.rename(columns={'date':'DATE','Tmin':'TMIN', 'Tmax':'TMAX'}, inplace=True)
weather_data["DATE"] = pd.to_datetime(weather_data["DATE"].astype(str), format='%Y-%m-%d')
weather_data = weather_data.sort_values(["DATE"]).reset_index(drop=True)

weather_data

## Setup initial parameters

In [3]:
# Initialization of variables 
params = dict(
    sowing_date = sowing_date, # Sowing date in YYYY-MM-DD
    latitude = lat, # Latitude of the site
    GDDE = 6.2, # Growing degree days per cm seed depth required for emergence, GDD/cm
    VREQ  = 505.0, # Vernalization required for max.development rate (VDays)
    PHINT = 95.0, # Phyllochron. A good estimate for PHINT is 95 degree days. This value for PHINT is appropriate except for spring sown wheat in latitudes greater than 30 degrees north and 30 degrees south, in which cases a value for PHINT of 75 degree days is suggested. 
    P1V = 1.85, # development genetic coefficients, vernalization. 1 for spring type, 5 for winter type
    P1D = 3.675, # development genetic coefficients, Photoperiod (1 - 6, low - high sensitive to day length)
)


**Note**: All the paramters above are given by default except for `sowing_date` and `latitude`. 

## Estimate Phenological stage

In [ ]:
%%time

growstages = determine_phenology_stage(initparams=params, weather=weather_data, dispDates=True, verbose=False)
